# Módulo 5: La Caja de Cristal (Interpretability & SHAP)

## Objetivo Didáctico
Convertir nuestro potente XGBoost (Caja Negra) en una **Caja de Cristal**. Usaremos **SHAP** (SHapley Additive exPlanations), una técnica ganadora del Premio Nobel de Economía (Teoría de Juegos), para explicar CADA predicción.

## 1. Preparando el Modelo
Primero, entrenamos nuestro XGBoost como siempre.

In [ ]:
# Instalar SHAP si no lo tienes
!pip install shap

In [ ]:
import pandas as pd
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Cargar datos
df = pd.read_csv('../../data/processed/clean_student_data.csv')
df_binary = df[df['Target'].isin(['Dropout', 'Graduate'])].copy()
df_binary['Target_Binary'] = df_binary['Target'].apply(lambda x: 1 if x == 'Dropout' else 0)

features = [
    'Curricular units 1st sem (grade)', 
    'Age at enrollment', 
    'Tuition fees up to date', 
    'Scholarship holder', 
    'Debtor'
]

X = df_binary[features]
y = df_binary['Target_Binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar XGBoost
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

## 2. Encendiendo la Luz (SHAP)
SHAP calcula cuánto contribuyó cada variable a la predicción, comparado con el promedio.

In [ ]:
# Inicializar JS para gráficos interactivos (si corres esto en local)
shap.initjs()

# Crear el explicador
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

## 3. Explicación Global (Resumen)
Este gráfico es **mucho mejor** que el de "Importancia de Variables" clásico.
*   **Color Rojo**: Valor alto de la variable (ej. Nota Alta).
*   **Color Azul**: Valor bajo de la variable (ej. Nota Baja).
*   **Lado Derecho**: Aumenta el riesgo de Dropout.
*   **Lado Izquierdo**: Disminuye el riesgo (Protege).

In [ ]:
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test, show=True)

### Lectura
Mira `Curricular units 1st sem (grade)`:
*   Los puntos **Rojos** (Notas Altas) están a la **Izquierda** (Bajan el riesgo).
*   Los puntos **Azules** (Notas Bajas) están a la **Derecha** (Suben el riesgo).

¡Ahora sí sabemos la dirección del impacto!

## 4. Explicación Local (Caso Individual)
¿Por qué el Estudiante #5 tiene un riesgo alto? Vamos a verlo.

In [ ]:
# Elegimos un estudiante al azar (índice 5)
idx = 5
shap.plots.waterfall(shap_values[idx])

### Lectura del Waterfall
*   **E[f(x)]**: Es el riesgo promedio de todos los alumnos (ej. 0.5).
*   **Barras Rojas**: Factores que empujaron su riesgo hacia arriba.
*   **Barras Azules**: Factores que lo empujaron hacia abajo.
*   **f(x)**: Es su riesgo final.

Esto te permite decirle a un directivo: "Este alumno tiene riesgo porque debe la matrícula (+20%) y reprobó el semestre (+15%), aunque su beca le ayudó un poco (-5%)".

## Conclusión Final
Hemos completado el viaje:
1.  **Regresión Logística**: Entendimos la base.
2.  **Árboles**: Entendimos las decisiones lógicas.
3.  **Random Forest**: Entendimos la estabilidad.
4.  **XGBoost**: Conseguimos la máxima potencia.
5.  **SHAP**: Recuperamos la explicabilidad.

¡Estás listo para tu Tesis!